# Project group 30

In [1]:
import numpy as np
import pandas as pd
pd.options.display.max_rows = 20
np.set_printoptions(precision = 4, suppress = True)

In [2]:
#pip install pycountry_convert
from pycountry_convert import country_name_to_country_alpha2

In [3]:
population = 'population.csv'
df_population= pd.read_csv(population)
medals = 'medals.csv'
df_medals= pd.read_csv(medals)
cpi = 'cpi.csv'
df_cpi= pd.read_csv(cpi)

#df_population

,Country (or dependency),Population (2020),Yearly Change,Net Change,Density (P/Km²),Land Area (Km²),Migrants (net),Fert. Rate,Med. Age,Urban Pop %,World Share
0,China,1440297825,0.39 %,5540090,153,9388211,-348399.0,1.7,38,61 %,18.47 %
1,India,1382345085,0.99 %,13586631,464,2973190,-532687.0,2.2,28,35 %,17.70 %
2,United States,331341050,0.59 %,1937734,36,9147420,954806.0,1.8,38,83 %,4.25 %
3,Indonesia,274021604,1.07 %,2898047,151,1811570,-98955.0,2.3,30,56 %,3.51 %
4,Pakistan,221612785,2.00 %,4327022,287,770880,-233379.0,3.6,23,35 %,2.83 %
...,...,...,...,...,...,...,...,...,...,...,...
230,Montserrat,4993,0.06 %,3,50,100,NaN,N.A.,N.A.,10 %,0.00 %
231,Falkland Islands,3497,3.05 %,103,0,12170,NaN,N.A.,N.A.,66 %,0.00 %
232,Niue,1628,0.68 %,11,6,260,NaN,N.A.,N.A.,46 %,0.00 %
233,Tokelau,1360,1.27 %,17,136,10,NaN,N.A.,N.A.,0 %,0.00 %


In [14]:
import pycountry
import pycountry_convert as pcc
from fuzzywuzzy import fuzz, process

# Create a list of all unique countries in the dataframe
countries = df_population['Country (or dependency)'].unique().tolist()
print(countries)

#Create a list of all standard country names
countries_standard = [country.name for country in pycountry.countries]
#print(countries_standard)

for i in range(len(df_population)):
    temp = df_population['Country (or dependency)'][i]
    country = 'c'
    try:
        country = pycountry.countries.lookup(temp)
        print(country)
    except LookupError as e:
        print(country)
        continue
    if country == 'c':
        #print('k')
        try:
            match = process.extractOne(temp, countries_standard, scorer=fuzz.token_sort_ratio)
            #print('kk')
            if match[1] > 80:
                country_code = pcc.country_name_to_country_alpha2(match[0])
                df_population.loc[i, 'Country (or dependency)'] = country_code
            else:
                df_population.loc[i, 'Country (or dependency)'] = None
        except ValueError as e:
            df_population.loc[i, 'Country (or dependency)'] = None
    else:
        df_population.loc[i, 'Country (or dependency)'] = country.alpha_2
    
print(df_population['Country (or dependency)'])


['CN', 'IN', 'US', 'ID', 'PK', 'BR', 'NG', 'BD', 'Russia', 'MX', 'JP', 'ET', 'PH', 'EG', 'VN', 'DR Congo', 'TR', 'Iran', 'DE', 'TH', 'GB', 'FR', 'IT', 'TZ', 'ZA', 'MM', 'KE', 'KR', 'CO', 'ES', 'UG', 'AR', 'DZ', 'SD', 'UA', 'IQ', 'AF', 'PL', 'CA', 'MA', 'SA', 'UZ', 'PE', 'AO', 'MY', 'MZ', 'GH', 'YE', 'NP', 'VE', 'MG', 'CM', 'CI', 'KP', 'AU', 'NE', 'TW', 'LK', 'BF', 'ML', 'RO', 'MW', 'CL', 'KZ', 'ZM', 'GT', 'EC', 'Syria', 'NL', 'SN', 'KH', 'TD', 'SO', 'ZW', 'GN', 'RW', 'BJ', 'BI', 'TN', 'BO', 'BE', 'HT', 'CU', 'SS', 'DO', 'Czech Republic (Czechia)', 'GR', 'JO', 'PT', 'AZ', 'SE', 'HN', 'AE', 'HU', 'TJ', 'BY', 'AT', 'PG', 'RS', 'IL', 'CH', 'TG', 'SL', 'HK', 'Laos', 'PY', 'BG', 'LY', 'LB', 'NI', 'KG', 'SV', 'TM', 'SG', 'DK', 'FI', 'CG', 'SK', 'NO', 'OM', 'State of Palestine', 'CR', 'LR', 'IE', 'CF', 'NZ', 'MR', 'PA', 'KW', 'HR', 'MD', 'GE', 'ER', 'UY', 'BA', 'MN', 'AM', 'JM', 'QA', 'AL', 'PR', 'LT', 'NA', 'GM', 'BW', 'GA', 'LS', 'MK', 'SI', 'GW', 'LV', 'BH', 'GQ', 'TT', 'EE', 'TL', 'MU', 'C